In [1]:
import torch
import torch.nn as nn
import torchvision
from torchvision import models, transforms
from PIL import Image
import os


In [3]:
# Load the Drive helper and mount
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# Vérifier si CUDA est disponible, sinon utiliser le CPU
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Recréer le modèle et charger les poids sauvegardés
model = models.densenet201(pretrained=True)

# Modifier la dernière couche du modèle comme tu l'as fait dans l'entraînement
classifier = nn.Sequential(nn.Linear(1920, 256),
                           nn.ReLU(),
                           nn.Linear(256, 2),
                           nn.LogSoftmax(dim=1))
model.classifier = classifier

# Charger les poids sauvegardés
model.load_state_dict(torch.load('/content/drive/My Drive/Coriander_vs_Parsley/coriander_vs_parsley_model_weights.pth'))

# Mettre le modèle en mode évaluation
model.eval()

# Envoyer le modèle sur le bon device (GPU ou CPU)
model.to(device)


<ipython-input-4-2dc39c2e1294>:15: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('/content/drive/My Drive/Coriander_vs_Parsley/coriander_vs_

DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace=True)
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu

In [5]:
def predict_image(image_path, model, transform, device):
    """
    Cette fonction charge une image, applique les transformations et fait une prédiction
    avec le modèle chargé.

    Args:
    - image_path: chemin vers l'image à prédire
    - model: le modèle chargé avec les poids
    - transform: transformations à appliquer sur l'image
    - device: le dispositif ('cuda' ou 'cpu')

    Retourne:
    - La classe prédite pour l'image.
    """
    # Charger l'image
    image = Image.open(image_path)

    # Appliquer les transformations sur l'image
    image = transform(image).unsqueeze(0)  # Ajouter une dimension batch

    # Envoyer l'image sur le bon device (GPU ou CPU)
    image = image.to(device)

    # Faire une prédiction avec le modèle
    with torch.no_grad():
        output = model(image)

    # Convertir les logits en probabilités
    ps = torch.exp(output)

    # Obtenir la classe prédite
    top_p, top_class = ps.topk(1, dim=1)

    return top_class.item()

# Transformer l'image avant la prédiction (en utilisant les mêmes transformations que pendant l'entraînement)
test_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])


In [7]:
# Chemin de l'image que tu veux tester
image_path = '/content/drive/My Drive/Coriander_vs_Parsley/5110_2.jpg'  # Remplace par ton chemin d'image

# Obtenir la classe prédite
predicted_class = predict_image(image_path, model, test_transforms, device)

# Liste des classes (Coriander et Parsley)
class_names = ['Coriander', 'Parsley']

# Afficher la classe prédite
predicted_label = class_names[predicted_class]
print(f'Classe prédite: {predicted_label}')


Classe prédite: Parsley
